## Comparing DR1 and GalaxyZoo datasets

In [9]:
import pandas as pd
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord

In [4]:
desi_path = '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg'
desi_cat = "../Data/gz_desi_deep_learning_catalog_friendly.parquet"
gz1_cat = "../Data/GalaxyZoo1_DR_table2.csv"

# read DESI catalogue:
desi_data = pd.read_parquet(desi_cat)
gz1_data = pd.read_csv(gz1_cat)
print(f"Number of galaxies in DESI catalogue: {len(desi_data)}")
print(f"Number of galaxies in GZ1 catalogue: {len(gz1_data)}")

Number of galaxies in DESI catalogue: 8689370
Number of galaxies in GZ1 catalogue: 667944


In [ ]:
def find_desi_galaxy_by_id(dr8_id, brick_id):
    galaxies = desi_data[ (desi_data['brickid']==brick_id) & (desi_data['dr8_id']==dr8_id) ]
    print(f"Found {len(galaxies)} matching.")
    print(galaxies)

def find_desi_galaxy_by_coords(ra, dec):
    galaxies = desi_data[ (desi_data['ra']==ra)& (desi_data['dec']==dec) ]
    print(f"Found {len(galaxies)} matching.")
    print(galaxies)

def find_gz1_galaxy_by_id(objid):
    galaxies = gz1_data[gz1_data['OBJID']==objid]
    print(f"Found {len(galaxies)} matching.")
    print(galaxies)

def find_gz1_galaxy_by_coords(ra, dec):
    galaxies = gz1_data[ (gz1_data['RA']==ra)& (gz1_data['DEC']==dec) ]
    print(f"Found {len(galaxies)} matching.")
    print(galaxies)

def get_filepath_by_id(dr8_id, brick_id):
    file_loc = f"{desi_path}/{brick_id}/{dr8_id}.jpg"
    return file_loc

# find_galaxy('100000_1401',100000)
# print(get_filepath('100000_1401',100000))
print(gz1_data.columns.values)
#desi_data.head(5)
gz1_data.head(5)

['OBJID' 'RA' 'DEC' 'NVOTE' 'P_EL' 'P_CW' 'P_ACW' 'P_EDGE' 'P_DK' 'P_MG'
 'P_CS' 'P_EL_DEBIASED' 'P_CS_DEBIASED' 'SPIRAL' 'ELLIPTICAL' 'UNCERTAIN']


In [14]:
ra1 = gz1_data['RA'][0]
dec1 = gz1_data['DEC'][1]
print(ra1,dec1)
c = SkyCoord(ra=ra1, dec=dec1, unit=u.degree)
print(c)

<SkyCoord (ICRS): (ra, dec) in deg
    (0.00011389, -9.22227778)>
